# **Importing the packages**

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import MultiLabelBinarizer

# **Loading the Datasets**

In [21]:
ratings_data = pd.read_csv('C:/Users/srika/OneDrive/Desktop/Bharat internship/Movie Recommendations/ratings.csv')

print("Ratings Data:")
print(ratings_data)

movies_data = pd.read_csv('C:/Users/srika/OneDrive/Desktop/Bharat internship/Movie Recommendations/movies.csv')

print("\nMovies Data:")
print(movies_data)

Ratings Data:
    userId  movieId  rating
0        1        1      64
1        2        2      68
2        3        3      43
3        4        4      15
4        5        5      28
..     ...      ...     ...
72      73       73      54
73      74       74      89
74      75       75      79
75      76       76      40
76      77       77      87

[77 rows x 3 columns]

Movies Data:
    movieId                               title   genres
0         1          Zack and Miri Make a Porno  Romance
1         2                     Youth in Revolt   Comedy
2         3  You Will Meet a Tall Dark Stranger   Comedy
3         4                        When in Rome   Comedy
4         5               What Happens in Vegas   Comedy
..      ...                                 ...      ...
72       73                 Across the Universe  romance
73       74                       A Serious Man    Drama
74       75                  A Dangerous Method    Drama
75       76                          27 Dre

# **Merge Data**

In [22]:
data = pd.merge(ratings_data, movies_data, on='movieId')

# **One-Hot Encode Genres**

In [23]:
genres_list = [genres.split('|') for genres in data['genres']]
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(genres_list), columns=mlb.classes_)
data = pd.concat([data, genres_encoded], axis=1)

# **Create Model**

In [24]:
genres_input = Input(shape=[len(mlb.classes_)], name='genres')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the model using only genre information
embedding_size = 30
dropout_rate = 0.2

genres_embedding = Embedding(input_dim=len(data['movieId'].unique()), output_dim=embedding_size, name='genres_embedding')(genres_input)
flattened_genres = Flatten()(genres_embedding)
dense_layer_1 = Dense(128, activation='relu')(flattened_genres)
dropout_layer = Dropout(dropout_rate)(dense_layer_1)
dense_layer_2 = Dense(64, activation='relu')(dropout_layer)
output_layer = Dense(1)(dense_layer_2)

model = Model(inputs=genres_input, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error')

# **Train Model**

In [25]:
model.fit(train_data[mlb.classes_], train_data['rating'], epochs=10, batch_size=64)

Epoch 1/10
1/1 [==============================] - 1s 665ms/step - loss: 3136.4966
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 3131.3157
Epoch 3/10
1/1 [==============================] - 0s 5ms/step - loss: 3126.7092
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 3121.4749
Epoch 5/10
1/1 [==============================] - 0s 5ms/step - loss: 3116.3604
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 3108.8274
Epoch 7/10
1/1 [==============================] - 0s 6ms/step - loss: 3102.7002
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 3095.0647
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 3085.7830
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 3078.6094


# **Evaluate Model**

In [26]:
model.fit(train_data[mlb.classes_], train_data['rating'], epochs=10, batch_size=64)

Epoch 1/10
1/1 [==============================] - 0s 8ms/step - loss: 3067.3494
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 3055.0449
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 3043.6174
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 3028.8059
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 3013.3210
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 2995.7268
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 2975.4856
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 2954.5093
Epoch 9/10
1/1 [==============================] - 0s 4ms/step - loss: 2934.4534
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 2909.2129


# **User Input for Genres**

In [27]:
user_genre = input("Enter a genre: ")

Enter a genre:  Comedy


# **Filter Movies**

In [28]:
filtered_movies = movies_data[movies_data['genres'].str.contains(user_genre, case=False)]

# **Make Recommendations**

In [29]:
if not filtered_movies.empty:
    # Get predictions for filtered movies
    filtered_movies_data = pd.merge(test_data, filtered_movies[['movieId']], on='movieId', how='inner')
    recommendations = model.predict(filtered_movies_data[mlb.classes_])

    # Combine movie indices and predictions
    recommendations_df = pd.DataFrame({'movieId': filtered_movies_data['movieId'], 'prediction': recommendations.squeeze()})

    # Display top N recommendations
    top_n_recommendations = recommendations_df.sort_values(by='prediction', ascending=False).head(5)
    top_n_movies = movies_data[movies_data['movieId'].isin(top_n_recommendations['movieId'])]
    print(f"\nTop 5 Movie Recommendations based on Genre '{user_genre}':\n")
    print(top_n_movies[['title', 'genres']])
else:
    print(f"No movies found for the genre '{user_genre}'.")

1/1 [==============================] - 0s 94ms/step

Top 5 Movie Recommendations based on Genre 'Comedy':

                         title  genres
4        What Happens in Vegas  Comedy
45                  Mamma Mia!  Comedy
50          Life as We Know It  Comedy
53                   Leap Year  Comedy
66  Ghosts of Girlfriends Past  Comedy
